In [2]:
##############
#Dependencies#
##############
from IPython.display import display, HTML
import os
import csv
import pandas as pd



#######
#Paths#
#######
filename = "Resources/purchase_data.csv"
output_path = os.path.join("Output", "GenderAnalysis.csv")
output_path2 = os.path.join("Output", "AgeAnalysis.csv")
output_path3 = os.path.join("Output", "TopSpenders.csv")
output_path4 = os.path.join("Output", "TopSellingItems.csv")
output_path5 = os.path.join("Output", "ProfitableItems.csv")

###################
#Initial Variables#
###################

names = []
ages = []
genders = []
itemids = []
itemnames = []
prices = []


males = 0
females = 0
other = 0 
malep = 0.0
femalep = 0.0
otherp = 0.0

uniqueplayers = []
uniqueitems = []


totalsales = 0
totalplayers = 0

uniqueitemsnum = 0
totalrevenue = 0.0
avgprice = 0.0


#######################################
#Load & Make Extra Copies of DataFrame#
#######################################

df = pd.read_csv(filename, header=0, index_col=0)
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()

#Create Dataframe df2 WITHOUT Repeat Players
df2 = df2.drop_duplicates(subset=['SN'])


#Dataframe Load Columns
names = df["SN"]
ages = df["Age"]
genders = df2["Gender"]
itemids = df["Item ID"]
itemnames = df["Item Name"]
prices = df["Price"]


#Create Series With Unique Player Names & Items
uniqueplayers = pd.Series(names).unique()  
uniqueplayers.sort()
uniqueitems = pd.Series(itemnames).unique()
uniqueitems.sort()

#Sales, Player, Revenue Totals & Average Price
totalsales = len(df.index)
totalplayers = len(df2)
uniqueitemsnum = len(uniqueitems)
totalrevenue = prices.sum()
avgprice = totalrevenue / totalsales

#Summary Gender Demographics (No Repeat Players)
genderlist = pd.Series(genders, dtype="string")
males = sum(genderlist.str.count('Male'))
females = sum(genderlist.str.count('Female'))
other = totalplayers - males - females
malep = (males / totalplayers) * 100
femalep = (females / totalplayers) * 100
otherp = (other / totalplayers) * 100


############################################
#Gender Demographics Using DataFrames 1 & 2#
############################################

#Males 
isMale = df['Gender']=='Male'
df_isMale = df[isMale]
malesales = len(df_isMale)
maleavg = (df_isMale['Price'].sum() / malesales)
malerevenue = df_isMale['Price'].sum()

#Create DataFrame of Sales to Male Players Sorted By Name in Alphabetical Order
males1 = df_isMale.sort_values(by=['SN'])
uniquemales2 = pd.Series(df_isMale['SN']).unique()   
uniquemales2.sort()
malenames = len(uniquemales2)
males1 = males1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
males1 = males1.reset_index(drop=True)

#Create Series of Sales to Male Players Indexed by Name
maleprice = pd.Series(males1["Price"].values, index=males1['SN'])
maleavg2 = (maleprice.groupby(level=0).sum()).sum() / malenames
  
    
#Females
isFemale = df['Gender']=='Female'
df_isFemale = df[isFemale]
femalesales = len(df_isFemale)
femaleavg = (df_isFemale['Price'].sum() / femalesales)
femalerevenue = df_isFemale['Price'].sum()

#Create DataFrame of Sales to Female Players Sorted By Name in Alphabetical Order
females1 = df_isFemale.sort_values(by=['SN'])
uniquefemales2 = pd.Series(df_isFemale['SN']).unique()   
uniquefemales2.sort()
femalenames = len(uniquefemales2)
females1 = females1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
females1 = females1.reset_index(drop=True)

#Create Series of Sales to Female Players Indexed by Name
femaleprice = pd.Series(females1["Price"].values, index=females1['SN'])
femaleavg2 = (femaleprice.groupby(level=0).sum()).sum() / femalenames

#Other/Non-Disclosed
isOther = df['Gender']=='Other / Non-Disclosed'
df_isOther = df[isOther]
othersales = len(df_isOther)
otheravg = (df_isOther['Price'].sum() / othersales)
otherrevenue = df_isOther['Price'].sum()

#Create DataFrame of Sales to Other Players Sorted By Name in Alphabetical Order
other1 = df_isOther.sort_values(by=['SN'])
uniqueother2 = pd.Series(df_isOther['SN']).unique()   
uniqueother2.sort()
othernames = len(uniqueother2)
other1 = other1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
other1 = other1.reset_index(drop=True)

#Create Series of Sales to Other Players Indexed by Name
otherprice = pd.Series(other1["Price"].values, index=other1['SN'])
otheravg2 = (otherprice.groupby(level=0).sum()).sum() / othernames

#####################################################
#Create Series of Total Sales Indexed by Player Name#
#####################################################

maleseries = maleprice.groupby(level=0).sum()
femaleseries = femaleprice.groupby(level=0).sum()
otherseries = otherprice.groupby(level=0).sum()

saleseries = maleseries.append(femaleseries)
saleseries = saleseries.append(otherseries)

####################################
#Age Demographics Using DataFrame 3#
####################################

df3 = df3.drop(columns=['Item ID', 'Item Name'])
df3 = df3.sort_values(by=['Age'])
df3 = df3.reset_index(drop=True)
agebins = pd.cut(df3['Age'], 9)

#Create Player & Price Series Indexed by Age Intervals
ageseries = pd.Series(df3["Price"].values, index=agebins)
nameseries = pd.Series(df3['SN'].values, index=agebins)

agesum = ageseries.groupby(level=0).sum()
ageavg = ageseries.groupby(level=0).mean()
agesales = nameseries.groupby(level=0).count()
agesales = agesales.reset_index(drop=True)
agetotal = agesales.array


#Remove Duplicates From Player Name Series
nameseries = nameseries.drop_duplicates()
agenum = nameseries.groupby(level=0).count()

ageavg2 = agesum / agenum


##################################
#Top 5 Spenders Using DataFrame 4#
##################################

#Sort the Series of $ Spent Per Player Indexed By Name, Create New Series With Only Top 5
saleseries = saleseries.sort_values(ascending=False)
topfive = saleseries.head()
spendnames = topfive.index.values
spendnames = spendnames.tolist()
topfive.reset_index(drop=True)
spendsum = topfive.array

#Create Series of Player Purchases Indexed by Name 
namelist1 = pd.Series(df4['Price'].values, index=df4['SN'])

#Create Series With All Sales of Top 5 Spenders 
namelist1 = namelist1.loc[spendnames]
spendavg = namelist1.groupby(level=0).mean()
#Count & Reorder Names
spendcounts = namelist1.index.value_counts()
spendcounts = spendcounts.reindex(index = spendnames)



######################################
#Most Popular Items Using DataFrame 5#
######################################


#Create Series of Purchases Indexed by Item Name
itemseries = pd.Series(df5['Price'].values, index=df5['Item Name'])

#Find Top 5 Bestsellers & Calculate Sales, Revenue, Mean Price
itemcounts = itemseries.index.value_counts()
itemcounts = itemcounts.head()
bestsellername = itemcounts.index.values
bestsellername = bestsellername.tolist()

itemseries = itemseries.loc[bestsellername]
bestsellertotal = itemseries.groupby(level=0).sum()
bestsellertotal = bestsellertotal.tolist()
bestsellerprice = itemseries.groupby(level=0).mean()
bestsellerprice = bestsellerprice.tolist()

#Create Series of Item IDs Indexed by Item Name, Retrieve Item IDs for Top 5 Bestselling
bestsellerid = pd.Series(df5['Item ID'].values, index=df5['Item Name'])
bestsellerid = bestsellerid.drop_duplicates()
bestsellerid = bestsellerid.reindex(index=bestsellername)
bestsellerid = bestsellerid.tolist()



#########################################
#Most Profitable Items Using DataFrame 6#
#########################################

#Create Purchase Series Indexed by Item Name
finalseries1 = pd.Series(df6['Price'].values, index=df6['Item Name'])
                                                        
#Create Profit Series Indexed By Item Name; Find Top 5 Sales & Revenue
profitseries = finalseries1.groupby(level=0).sum()
profitseries = profitseries.sort_values(ascending=False)
profitseries = profitseries.head()
profitname = profitseries.index.values
profitname = profitname.tolist()
profitrevenue = profitseries.values
profitrevenue = profitrevenue.tolist()
finalseries1 = finalseries1.loc[profitname]

#Count Sales & Calculate Mean Price
profitcount = finalseries1.index.value_counts()
profitprice = finalseries1.groupby(level=0).mean()

#Create Series of Item IDs Indexed by Item Name, Retrieve IDs for Top 5 Profitable
profitid = pd.Series(df6['Item ID'].values, index=df6['Item Name'])
profitid = profitid.drop_duplicates()
profitid = profitid.reindex(index=profitname)
profitid = profitid.tolist()


##############################################
#Generate Text Report With Summary Statistics#
##############################################

f = open("Output/Summary.txt", "a")

f.write("\n" + "\n" + "Summary Stats \n")
f.write("Total Sales : " + str(totalsales) + "\n")
f.write("Total Players: " + str(totalplayers) + "\n")
f.write("Unique Items: " + str(uniqueitemsnum) + "\n")
f.write("Total Revenue: $" + str(totalrevenue) + "\n")
f.write("Average Purchase: $" + str(avgprice) + "\n")

f.write("Conclusions: \n")
f.write("The overwhelming majority of gamers who make purchases are male.")
f.write("There is little range in the average purchase between players, the average purchase for all players is less than $2 less than the highest spending players.")
f.write("Final Critic, Oathbreaker, and Nirvana are the 3 most succesful items sold by the game thus far, being both top 3 in bestselling and most profitable.")

f.close()

############################
#Load Data Into DataFrames#
###########################

#Gender
genderdata = [(males, females, other), (malep, femalep, otherp), (malesales, femalesales, othersales), (malerevenue, femalerevenue, otherrevenue),
             (maleavg, femaleavg, otheravg), (maleavg2, femaleavg2, otheravg2)]

dfgender = pd.DataFrame(genderdata, columns = ['Male' , 'Female', 'Other'], index=['No. Players', '% Players', 'No. Sales','$ Total' , '$ Average Sale', '$ Average Per Player'])

#Age
agedata = [agetotal, ageavg, agesum, ageavg2]

dfage = pd.DataFrame(agedata, columns = ['7-11 yrs', '12-15 yrs', '16-19 yrs', '20-23 yrs', '24-28 yrs', '29-32 yrs', '33-36 yrs', '37-40 yrs', '41-45 yrs' ], index=['No. Sales', '$ Avg Sale', 'Total $', '$ Avg Per Person'])

#Top 5 Spenders
spenderdata = [spendnames, spendcounts, spendavg, spendsum]

dfspender = pd.DataFrame(spenderdata, columns = ['#1', '#2', '#3', '#4', '#5'], index=['SN', 'Purchases', 'Avg $ Purchase', '$ Spent'] )

#Top 5 Best Selling

itemdata = [bestsellerid, bestsellername, itemcounts, bestsellerprice, bestsellertotal]

dfbestsellers = pd.DataFrame(itemdata, columns = ['#1', '#2', '#3', '#4', '#5'], index=['Item ID', 'Item Name', 'No. Sales', 'Price $', 'Revenue $' ])

#Top 5 Profitable

profitdata = [profitid, profitname, profitcount, profitprice, profitrevenue]

dfprofit = pd.DataFrame(profitdata, columns = ['#1', '#2', '#3', '#4', '#5'], index=['Item ID', 'Item Name', 'No. Sales', 'Price $', 'Revenue $' ])

########
#Output#
########

print("\n" + "\n" + "Summary Stats \n")
print("Total Sales : " + str(totalsales) + "\n")
print("Total Players: " + str(totalplayers) + "\n")
print("Unique Items: " + str(uniqueitemsnum) + "\n")
print("Total Revenue: $" + str(totalrevenue) + "\n")
print("Average Purchase: $" + str(avgprice) + "\n")
print("Conclusions: \n")
print("The overwhelming majority of gamers who make purchases are male.")
print("There is little range in the average purchase between players, the average purchase for all players is less than $2 less than the highest spending players.")
print("Final Critic, Oathbreaker, and Nirvana are the 3 most succesful items sold by the game thus far, being both top 3 in bestselling and most profitable.")

display(HTML(dfgender.to_html()))

display(HTML(dfage.to_html()))

display(HTML(dfspender.to_html()))

display(HTML(dfbestsellers.to_html()))

display(HTML(dfprofit.to_html()))

dfgender.to_csv(output_path)
dfage.to_csv(output_path2)
dfspender.to_csv(output_path3)
dfbestsellers.to_csv(output_path4)
dfprofit.to_csv(output_path5)



Summary Stats 

Total Sales : 780

Total Players: 576

Unique Items: 179

Total Revenue: $2379.77

Average Purchase: $3.0509871794871795

Conclusions: 

The overwhelming majority of gamers who make purchases are male.
There is little range in the average purchase between players, the average purchase for all players is less than $2 less than the highest spending players.
Final Critic, Oathbreaker, and Nirvana are the 3 most succesful items sold by the game thus far, being both top 3 in bestselling and most profitable.


,Male,Female,Other
No. Players,484.000000,81.000000,11.000000
% Players,84.027778,14.062500,1.909722
No. Sales,652.000000,113.000000,15.000000
$ Total,1967.640000,361.940000,50.190000
$ Average Sale,3.017853,3.203009,3.346000
$ Average Per Player,4.065372,4.468395,4.562727


,7-11 yrs,12-15 yrs,16-19 yrs,20-23 yrs,24-28 yrs,29-32 yrs,33-36 yrs,37-40 yrs,41-45 yrs
No. Sales,39.000000,47.000000,101.000000,298.00000,155.000000,63.000000,42.000000,28.000000,7.000000
$ Avg Sale,3.275641,2.932128,3.041980,3.03302,3.019290,3.052222,2.895952,3.560357,3.075714
Total $,127.750000,137.810000,307.240000,903.84000,467.990000,192.290000,121.630000,99.690000,21.530000
$ Avg Per Person,4.258333,3.937429,3.793086,4.30400,4.069478,4.180217,3.923548,4.747143,3.075714


,#1,#2,#3,#4,#5
SN,Lisosia93,Idastidru52,Chamjask73,Iral74,Iskadarya95
Purchases,5,4,3,4,3
Avg $ Purchase,4.61,3.8625,3.405,4.36667,3.792
$ Spent,18.96,15.45,13.83,13.62,13.1


,#1,#2,#3,#4,#5
Item ID,92,178,82,132,145
Item Name,Final Critic,"Oathbreaker, Last Hope of the Breaking Storm",Nirvana,Persuasion,Fiery Glass Crusader
No. Sales,13,12,9,9,9
Price $,4.58,4.61462,4.9,4.23,3.22111
Revenue $,41.22,59.99,44.1,50.76,28.99


,#1,#2,#3,#4,#5
Item ID,92,178,82,145,103
Item Name,Final Critic,"Oathbreaker, Last Hope of the Breaking Storm",Nirvana,Fiery Glass Crusader,Singed Scalpel
No. Sales,13,12,9,9,8
Price $,4.58,4.61462,4.9,4.23,4.35
Revenue $,59.99,50.76,44.1,41.22,34.8
